In [1]:
from bs4 import BeautifulSoup
import requests
import time, os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Safari()
driver.set_window_size(1366,1024)

In [2]:
import pandas as pd
import sqlite3
import sys

In [3]:
driver.get('https://discoveruni.gov.uk/course-finder/results/')

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions   

In [ ]:
# SCRAPE ALL COURSE HREFS
hrefs_connection = sqlite3.connect('hrefs.db')

is_new_page = True
while is_new_page:
    # GET NEW PAGE SOURCE & SCRAPE ALL COURSE HREFS
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page_courses = soup.find_all('a', class_='course-finder-results__result-accordion-body-course-name')
    page_courses_href = [x.get('href') for x in page_courses]
    
    # MERGE ONTO DATABASE
    page_courses_href_df = pd.DataFrame(page_courses_href,columns=['href'])
    page_courses_href_df.to_sql('hrefs', hrefs_connection, if_exists='append', index=False)
    
    # IF NEXT PAGE EXISTS -> LOAD NEXT PAGE & REPEAT
    next_page = driver.find_elements_by_xpath('//a[@aria-label="Next page"][@class="pagination__link"]')
    is_new_page = bool(next_page)
    if is_new_page:
        next_page_number = str(int(driver.find_elements_by_xpath('//li[@class="pagination__item--active"]')[0].text)+1)
        next_page[0].click()
        # WAIT FOR NEXT PAGE TO LOAD BEFORE SCRAPING
        WebDriverWait(driver, 20).until(
            expected_conditions.text_to_be_present_in_element(
                (By.CLASS_NAME, 'pagination__item--active'),next_page_number
            )
        )

hrefs_connection.close()

In [3]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///hrefs.db")
hrefs = pd.read_sql('''
SELECT * FROM hrefs;''', engine)
hrefs

,href
0,/course-details/10008071/AAUNDERGRADUATE5YEAR/...
1,/course-details/10000047/PSSFDOPTDIS/Full-time
2,/course-details/10007783/NN24/Full-time
3,/course-details/10007783/NL41/Full-time
4,/course-details/10007783/NN34/Full-time
...,...
35208,/course-details/10007713/U1BATODMW/Full-time
35209,/course-details/10007713/U1BATODMS/Full-time
35210,/course-details/10007713/U1BAWSHIJF/Full-time
35211,/course-details/10007713/U1BAWSHIJ/Full-time


In [10]:
# hrefs['institution'] = hrefs['href'].apply(lambda x: x.split('/')[2])
# institutions_df = hrefs['institution'].drop_duplicates().reset_index().drop(['index'],axis=1)
engine = create_engine("sqlite:///institutions.db")
institutions_df = pd.read_sql('''
SELECT * FROM institutions;''', engine)
institutions_df

,institution,name,postcode,longitude,latitude
0,10008071,AA School of Architecture,WC1B3ES,-0.130697,51.518218
1,10000047,ABDO College of Education,CT47DT,0.952170,51.219855
2,10007783,University of Aberdeen,AB243FX,-2.099122,57.165019
3,10007849,Abertay University,DD11HG,-2.974299,56.463117
4,10007856,Aberystwyth University,SY233BF,-4.067570,52.417469
...,...,...,...,...,...
437,10008914,Coleg Y Cymoedd,CF157QX,-3.276072,51.565596
438,10007696,Yeovil College,BA214DR,-2.637780,50.948522
439,10007167,University of York,YO105DD,-1.053544,53.948419
440,10007709,York College,YO232BB,-1.113609,53.929705


In [ ]:
def get_institution_details(institution_id):
    url = 'https://discoveruni.gov.uk/institution-details/{}/'.format(institution_id)
    page = requests.get(url).text
    soup = BeautifulSoup(page, "html5lib")
    name = soup.find('h1', class_='institution-details__name').text.strip()
    postcode = soup.find('p', class_='institution-details__overview-item-value').text.split(',')[-1].strip().replace(' ','')
    return name, postcode
  
institutions_df['name'],  institutions_df['postcode'] = zip(*institutions_df['institution'].apply(lambda x: get_institution_details(x)))


In [ ]:
institutions_df

In [19]:
institutions_connection = sqlite3.connect('institutions.db')
institutions_df.to_sql('institutions', institutions_connection, if_exists='append', index=False)
institutions_connection.close()

In [78]:
import postcodes_io_api
import numpy as np

api  = postcodes_io_api.Api();



def get_institution_location(postcode):
    longitude = np.nan
    latitude = np.nan
    try:
        data = api.get_postcode(postcode).get('result')
        longitude = data.get('longitude')
        latitude = data.get('latitude')
        area = data.get('admin_district')
    except:
        longitude = np.nan
        latitude = np.nan
    return longitude, latitude, area

institutions_df['longitude'],  institutions_df['latitude'], institutions_df['area'] = zip(*institutions_df['postcode'].apply(lambda x: get_institution_coordinates(x)))




DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): api.postcodes.io:80
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/WC1B3ES HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/CT47DT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/AB243FX HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/DD11HG HTTP/1.1" 200 None


send: b'GET /postcodes/WC1B3ES HTTP/1.1\r\nHost: api.postcodes.io\r\nUser-Agent: python-requests/2.24.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 06 Jul 2021 13:54:34 GMT
header: Content-Type: application/json; charset=utf-8
header: Transfer-Encoding: chunked
header: Connection: keep-alive
header: x-gnu: Michael J Blanchard
header: access-control-allow-origin: *
header: etag: W/"34c-TbYrHeqg4PjSGRIjqsCf/9aQtUc"
header: CF-Cache-Status: HIT
header: Age: 79619
header: Report-To: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v2?s=kzQUNfafcMxIH4m1J4b6%2B4oyRfTZhqfPKMb%2FkpapbiTWEDlMGNHmwWQ2eShmQzhcMGXwDLgysm5V5nHy5u4i2H4fHQ3IqxkaqEBqnOu9SKQjgqf7BCOqm6%2FrEXoY"}],"group":"cf-nel","max_age":604800}
header: NEL: {"report_to":"cf-nel","max_age":604800}
header: Vary: Accept-Encoding
header: Server: cloudflare
header: CF-RAY: 66a953642c21ee03-CDG
header: Content-Encoding: gzip
send: b'GET /p

DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/SY233BF HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/OX141GG HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/SW183SX HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/GU14SB HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/OX11SA HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/BH52DF HTTP/1.1" 200 None


reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 06 Jul 2021 13:54:34 GMT
header: Content-Type: application/json; charset=utf-8
header: Transfer-Encoding: chunked
header: Connection: keep-alive
header: x-gnu: Michael J Blanchard
header: access-control-allow-origin: *
header: etag: W/"380-exv4O9wFxThF2ARI94UkGT7sac0"
header: CF-Cache-Status: HIT
header: Age: 236
header: Report-To: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v2?s=pXaW8lUZx7f4ZgJYc3iYy6nKmEvVpwTrfKmR4IeM1HgC6YLqFINy8uKHeYiLDVwmToy%2BZi8fXU9XN7JeovvNrt6BC9t6Iy3W17KDlG7KgwL5bUH7Y0OUzaogABg2"}],"group":"cf-nel","max_age":604800}
header: NEL: {"report_to":"cf-nel","max_age":604800}
header: Vary: Accept-Encoding
header: Server: cloudflare
header: CF-RAY: 66a953653e88ee03-CDG
header: Content-Encoding: gzip
send: b'GET /postcodes/OX141GG HTTP/1.1\r\nHost: api.postcodes.io\r\nUser-Agent: python-requests/2.24.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200

DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/SG128LX HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/WC1B3HN HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/CM11SQ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/CV34FJ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/W41LY HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/WC1V7EY HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/BH125HH HTTP/1.1" 200 None


...

send: b'GET /postcodes/CH414NT HTTP/1.1\r\nHost: api.postcodes.io\r\nUser-Agent: python-requests/2.24.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 06 Jul 2021 13:54:51 GMT
header: Content-Type: application/json; charset=utf-8
header: Transfer-Encoding: chunked
header: Connection: keep-alive
header: x-gnu: Michael J Blanchard
header: access-control-allow-origin: *
header: etag: W/"342-VSfQcGQngDqCVtw7ReLi9F2E48E"
header: CF-Cache-Status: HIT
header: Report-To: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v2?s=RB%2BYLMtXp14lETK5lNUSm5Sj6DcAMwzNLIZzt1WBTGUgvAZqj8BajkUAwI9K6qTcQxpnwJft3o%2FdwhPBeFfZFx4rDUmi0Yy6MXVjFhjMwb6F6KqZg%2BHOobTn%2Bv9K"}],"group":"cf-nel","max_age":604800}
header: NEL: {"report_to":"cf-nel","max_age":604800}
header: Vary: Accept-Encoding
header: Server: cloudflare
header: CF-RAY: 66a953d26ca0ee03-CDG
header: Content-Encoding: gzip
send: b'GET /postcodes/WV11LY H

DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/CM13RR HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/M145RB HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/CF157QX HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/BA214DR HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/YO105DD HTTP/1.1" 200 None


reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 06 Jul 2021 13:54:52 GMT
header: Content-Type: application/json; charset=utf-8
header: Transfer-Encoding: chunked
header: Connection: keep-alive
header: x-gnu: Michael J Blanchard
header: access-control-allow-origin: *
header: etag: W/"34e-YCVgPbuCsmmFlUE+5xfTCAqwXBw"
header: CF-Cache-Status: HIT
header: Report-To: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v2?s=uMLPpgjyaJfnAGi5Zs7zESxZuL7lX88fK%2BDzymqeOlwg%2B22cqLf5jtP665lHZNhJ65PHRN0oihObw66Sg5uLpchuZqUMWN77jj4yTGUuotVd0IuzR%2BVUohpG5zZY"}],"group":"cf-nel","max_age":604800}
header: NEL: {"report_to":"cf-nel","max_age":604800}
header: Vary: Accept-Encoding
header: Server: cloudflare
header: CF-RAY: 66a953d3af29ee03-CDG
header: Content-Encoding: gzip
send: b'GET /postcodes/M145RB HTTP/1.1\r\nHost: api.postcodes.io\r\nUser-Agent: python-requests/2.24.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
heade

DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/YO232BB HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://api.postcodes.io:80 "GET /postcodes/YO317EX HTTP/1.1" 200 None


reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 06 Jul 2021 13:54:52 GMT
header: Content-Type: application/json; charset=utf-8
header: Transfer-Encoding: chunked
header: Connection: keep-alive
header: X-GNU: Michael J Blanchard
header: Access-Control-Allow-Origin: *
header: ETag: W/"378-Je0g96RBMx7wIjrV5rN20XP5nJw"
header: CF-Cache-Status: HIT
header: Age: 879404
header: Report-To: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v2?s=4C55AN5QNke7aHEeWmIyIqWY3cqEsX2AQrM0u9IiOm0YWD6tx70bHoxAoDw3fKFj1XTKnlr4Cz4tSR7BnVRPI8i1AoVp%2FjiiJnoGPGLdQkpWzIsDOjoiyiP%2B5KEl"}],"group":"cf-nel","max_age":604800}
header: NEL: {"report_to":"cf-nel","max_age":604800}
header: Vary: Accept-Encoding
header: Server: cloudflare
header: CF-RAY: 66a953d4fa21ee03-CDG
header: Content-Encoding: gzip
send: b'GET /postcodes/YO317EX HTTP/1.1\r\nHost: api.postcodes.io\r\nUser-Agent: python-requests/2.24.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.

In [14]:
import postcodes_io_api
import numpy as np

api  = postcodes_io_api.Api();


def get_institution_area(postcode):
    longitude = np.nan
    latitude = np.nan
    try:
        data = api.get_postcode(postcode).get('result')
        area = data.get('admin_district')
    except:
        area = np.nan
    return area

institutions_df['area'] = institutions_df['postcode'].apply(lambda x: get_institution_area(x))

In [18]:
# institutions_df.drop(['longitude','latitude'],axis=1,inplace=True)
institutions_df[institutions_df['area'].isna()]

,institution,name,postcode,longitude,latitude,area
51,10008119,Bridgend College,,NaN,NaN,NaN
54,99999999,Brighton and Sussex Medical School,,NaN,NaN,NaN
185,99999998,Hull York Medical School,,NaN,NaN,NaN
251,99999997,Medway School of Pharmacy,,NaN,NaN,NaN
258,10045289,Montessori Centre International,NW80PJ,NaN,NaN,NaN
261,10004442,Moulton College,NN37RR,-0.857756,52.289955,None
281,10007138,The University of Northampton,NN15PH,-0.890179,52.231606,None
322,10005470,"Richmond, The American International Universit...",,NaN,NaN,NaN
